1. Einrichtung

In [ ]:
# Umgebung/Spiel installieren
!pip install gym_super_mario_bros==7.3.0 nes_py

In [ ]:
# Spiel importieren
import gym_super_mario_bros
# Joypad wrapper importieren
from nes_py.wrappers import JoypadSpace
# Vereinfachte Kontrolle importieren
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from gym_super_mario_bros.actions import COMPLEX_MOVEMENT
from gym_super_mario_bros.actions import RIGHT_ONLY

In [ ]:
##mögliche Action Lists und Ausgabe
RIGHT_ONLY

In [ ]:
COMPLEX_MOVEMENT

In [ ]:
SIMPLE_MOVEMENT

2. Umgebung einrichten

In [ ]:
# pytorch installieren
!pip install torch torchvision torchaudio

In [ ]:
# stable-baselines installieren
!pip install stable-baselines3[extra]

In [ ]:
# Grayscaling
from gym.wrappers import GrayScaleObservation

from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

from matplotlib import pyplot as plt

In [ ]:
# 1. Umgebung erstellen
env = gym_super_mario_bros.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode="human")
# 2. Vereinfachte Aktionen einführen 
env = JoypadSpace(env, SIMPLE_MOVEMENT)
JoypadSpace.reset = lambda self, **kwargs: self.env.reset(**kwargs)
# 3. Grayscaling anwenden
env = GrayScaleObservation(env, keep_dim=True)
# 4. Anwenden in Dummy-Umgebung
env = DummyVecEnv([lambda: env])
# 5. Frames Stapeln
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
#Momentaufnahme der Umgebung
state = env.reset()

In [ ]:
# Beispiel wie eine Aktion ausgeführt werden kann
state, reward, done, info = env.step([5])

In [ ]:
#Form der Umgebung
state.shape

In [ ]:
#Ausgabe der Momentaufnahme
plt.imshow(state[0])

2. Modell und Training einrichten

In [ ]:
# OS importieren zum Dokumentenmanagement
import os 
# A2C-Algortihmus importieren
from stable_baselines3 import A2C
# BaseCallback importieren um später Dokumentensicherung durch Callback zu programmieren
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
# Callback zur Dokumentensicherung
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
# Speicherorte 
CHECKPOINT_DIR = './train3/'
LOG_DIR = './logs3/'

In [ ]:
# Sicherung der Trainings- und Logdateien
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR))

In [ ]:
# Einrichtung des Agenten/Modells in der definierten Umgebung
model = A2C('CNNPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, 
            n_steps=512) 

In [ ]:
# Training des Agenten
model.learn(total_timesteps=5000000, callback=callback)

In [ ]:
# Agenten speichern
model.save('A2Cmodel')

4. Testen

In [ ]:
# Laden des Modells
model = A2C.load('./train2/best_model_5000000')

In [ ]:
#Testen der KI
state = env.reset()

while True: 
    # Vorraussage des Modells
    action, _ = model.predict(state)
    # Anwendung auf Umgebung
    state, reward, done, info = env.step(action)
    # Live-Ausgabe des Spiels
    env.render()

In [ ]:
# Umgebnung schließen
env.close()